In [152]:
import geopandas as gpd
import pandas as pd

In [153]:
gdf = gpd.read_file("all_states_precincts_cname_neighbor_demo_OG_anomalous_fixNeighborVA.json")
copy = gdf.copy()

In [154]:
gdf.head(1)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
0,Accomack,0001,486,637,16,8,4,2328,va,va-accomack-0001,...,3526,2336,906,12,20,246,None,None,None,"POLYGON ((-75.66233 37.77053, -75.66244 37.770..."


In [155]:
len(gdf)

11947

In [156]:
vaCong16 = pd.read_csv("VA/voting/congressional/VA_Congressional2016_Cleaned_Mapped_Final.csv")
vaCong18 = pd.read_csv("VA/voting/congressional/VA_Congressional2018_Cleaned_Mapped_Final.csv")
riCong16 = pd.read_csv("RI/voting/congresional/RI_Congressional_2016.csv")
riCong18 = pd.read_csv("RI/voting/congresional/RI_Congressional_2018.csv")
txCong16 = pd.read_csv("TX/voting/congressional/TX_Congressional_2016_Cleaned.csv")
txCong18 = pd.read_csv("TX/voting/congressional/TX_Congressional_2018_Cleaned.csv")

In [157]:
# add independent to  all others in vaCong
vaDFs = [vaCong16, vaCong18]
for df in vaDFs:
    for index, row in df.iterrows():
        df.at[index, "All Others"] = row["All Others"] + row["Independent"]
    df.drop(columns=["Independent"], axis=1, inplace=True)

In [158]:
mapper16 = {"Republican": "Cong16Rep", "Democratic": "Cong16Dem", "Libertarian": "Cong16Lib", "Green":"Cong16Grn"}
mapper18 = {"Republican": "Cong18Rep", "Democratic": "Cong18Dem", "Libertarian": "Cong18Lib", "Green":"Cong18Grn"}
vaCong16.rename(mapper16, inplace=True, axis=1)
vaCong18.rename(mapper18, inplace=True, axis=1)

In [159]:
mapper = {"Cong18Indep": "Cong18Grn"}
txCong18.rename(mapper, inplace=True, axis=1)

In [160]:
mapper16 = {"Independent":"Cong16Lib", "Republican": "Cong16Rep", "Democratic": "Cong16Dem"}
mapper18 = {"Independent":"Cong18Lib", "Republican": "Cong18Rep", "Democratic": "Cong18Dem"}
riCong16.rename(mapper16, inplace=True, axis=1)
riCong18.rename(mapper18, inplace=True, axis=1)
riCong16["Cong16Grn"] = 0
riCong18["Cong18Grn"] = 0

# texas cong18 is missing a cname

In [161]:
c1 = list(txCong16["CName"].unique())
c2 = list(txCong18["CName"].unique())
cgdf = list(gdf[gdf["State"] == "tx"]["CName"].unique())

In [162]:
for cname in c1:
    if cname not in c2:
        print(cname)

tx-angelina-15


In [163]:
for cname in c2:
    if cname not in c1:
        print(cname)

tx-harris-845


In [164]:
for cname in cgdf:
    if cname not in c1:
        print(cname)

tx-harris-845


In [165]:
for cname in cgdf:
    if cname not in c2:
        print(cname)

tx-angelina-15


In [166]:
print(len(c1), len(c2), len(txCong16), len(txCong18))

9081 9081 9082 9081


In [167]:
# txCong16 got duplicates
txCong16[txCong16.duplicated(["CName"])]

,dist,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,Type
4668,31,0.0,0.0,0.0,0.0,0.0,0.0,tx-bell-405,CONG16


In [168]:
txCong16.drop(index=4668, axis=0, inplace=True)

In [169]:
# add tx-angelina-15 to txCong18
txCong18[txCong18["CName"].str.contains("tx-angelina-14")]

,dist,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Grn,CName,Type
7,1,295,766,14,0,1075,0.0,tx-angelina-14,CONG18


In [170]:
txCong18 = txCong18.append({"dist": "1", "Cong18Dem":0, "Cong18Rep":0, "Cong18Lib":0,
                "Cong18Other":0, "Cong18Total": 0, "Cong18Grn":0,
                 "CName":"tx-angelina-15", "Type":"CONG18"}, ignore_index=True)
len(txCong18)

9082

In [171]:
# add tx-harris-845 to txCong16
txCong16[txCong16["CName"].str.contains("tx-harris-846")]

,dist,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,Type
2738,18,76.0,1.0,0.0,3.0,0.0,80.0,tx-harris-846,CONG16


In [172]:
txCong16 = txCong16.append({"dist": "29", "Cong16Dem":0, "Cong16Rep":0, "Cong16Lib":0,
                "Cong16Other":0, "Cong16Total": 0, "Cong16Grn":0,
                 "CName":"tx-harris-845", "Type":"CONG16"}, ignore_index=True)
len(txCong16)

9082

In [173]:
c1 = list(txCong16["CName"].unique())
c2 = list(txCong18["CName"].unique())
cgdf = list(gdf[gdf["State"] == "tx"]["CName"].unique())

In [174]:
for cname in cgdf:
    if cname not in c1:
        print(cname)

In [175]:
for cname in cgdf:
    if cname not in c2:
        print(cname)

# after fixing tx, save the version

In [176]:
vaCong16.to_csv("VA/voting/congressional/VA_Congressional2016_Cleaned_Mapped_Finalv2.csv", index=False)
vaCong18.to_csv("VA/voting/congressional/VA_Congressional2018_Cleaned_Mapped_Finalv2.csv", index=False)
riCong16.to_csv("RI/voting/congresional/RI_Congressional_2016v2.csv", index=False)
riCong18.to_csv("RI/voting/congresional/RI_Congressional_2018v2.csv", index=False)
txCong16.to_csv("TX/voting/congressional/TX_Congressional_2016_Cleanedv2.csv", index=False)
txCong18.to_csv("TX/voting/congressional/TX_Congressional_2018_Cleanedv2.csv", index=False)

# merge cong data

In [177]:
def merge(cong16, cong18):
    return cong16.merge(cong18, left_on="CName", right_on="CName")

In [178]:
riMerged = merge(riCong16, riCong18)
riMerged.head(1)

,Cong16Dem,Cong16Rep,Cong16Lib,Other_x,Total_x,dist_num_x,CName,Type_x,Cong16Grn,Cong18Dem,Cong18Rep,Cong18Lib,Other_y,Total_y,dist_num_y,Type_y,Cong18Grn
0,955,627,0,3,1691,1,ri-bristol-0101,CONG16,0,916,494,0,2,1412,1,CONG18,0


In [179]:
mapper = {"Other_x":"Cong16Other", "Total_x":"Cong16Total", "Other_y":"Cong18Other", "Total_y":"Cong18Total", "dist_num_y":"dist_num"}
riMerged.rename(mapper, axis=1, inplace=True)
riMerged.drop(columns=["dist_num_x", "Type_x", "Type_y"], axis=1, inplace=True)
riMerged.head(2)

,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Other,Cong16Total,CName,Cong16Grn,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,dist_num,Cong18Grn
0,955,627,0,3,1691,ri-bristol-0101,0,916,494,0,2,1412,1,0
1,1225,790,0,3,2167,ri-bristol-0102,0,1245,558,0,8,1811,1,0


In [180]:
vaMerged = merge(vaCong16, vaCong18)
vaMerged.head(1)

,Cong16Rep,Cong16Dem,Cong16Lib,Cong16Grn,All Others_x,Total Votes Cast_x,dist_num_x,CName,Type_x,Cong18Rep,Cong18Dem,Cong18Lib,Cong18Grn,All Others_y,Total Votes Cast_y,dist_num_y,Type_y
0,712,395,0,0,0,1107,2,va-accomack-0001,CONG16,533,407,0,0,0,940,2,CONG18


In [181]:
mapper = {"All Others_x":"Cong16Other", "Total Votes Cast_x":"Cong16Total", "All Others_y":"Cong18Other", "Total Votes Cast_y":"Cong18Total", "dist_num_y":"dist_num"}
vaMerged.rename(mapper, axis=1, inplace=True)
vaMerged.drop(columns=["dist_num_x", "Type_x", "Type_y"], axis=1, inplace=True)
vaMerged.head(2)

,Cong16Rep,Cong16Dem,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,Cong18Rep,Cong18Dem,Cong18Lib,Cong18Grn,Cong18Other,Cong18Total,dist_num
0,712,395,0,0,0,1107,va-accomack-0001,533,407,0,0,0,940,2
1,595,126,0,0,2,723,va-accomack-0002,452,149,0,0,2,603,2


In [182]:
txMerged = merge(txCong16, txCong18)
txMerged.head(1)

,dist_x,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,Type_x,dist_y,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Grn,Type_y
0,1,106.0,162.0,3.0,0.0,0.0,271.0,tx-angelina-1,CONG16,1,86,129,3,0,218,0.0,CONG18


In [183]:
mapper = {"dist_y":"dist_num"}
txMerged.rename(mapper, axis=1, inplace=True)
txMerged.drop(columns=["dist_x", "Type_x", "Type_y"], axis=1, inplace=True)
txMerged.head(2)

,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Grn,Cong16Other,Cong16Total,CName,dist_num,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,Cong18Grn
0,106.0,162.0,3.0,0.0,0.0,271.0,tx-angelina-1,1,86,129,3,0,218,0.0
1,261.0,352.0,10.0,0.0,0.0,623.0,tx-angelina-0010a,1,193,229,3,0,425,0.0


In [184]:
all_merged = [riMerged, vaMerged, txMerged]
all_state_merged = pd.concat(all_merged, ignore_index=True)
all_state_merged.head(1)

,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Other,Cong16Total,CName,Cong16Grn,Cong18Dem,Cong18Rep,Cong18Lib,Cong18Other,Cong18Total,dist_num,Cong18Grn
0,955.0,627.0,0.0,3.0,1691.0,ri-bristol-0101,0.0,916,494,0,2,1412,1,0.0


# check if all cname in merged is also in gdf and vice versa

In [185]:
c1 = list(gdf["CName"].unique())
c2 = list(all_state_merged["CName"].unique())

In [186]:
for cname in c1:
    if cname not in c2:
        print(cname)

In [187]:
for cname in c2:
    if cname not in c1:
        print(cname)

In [188]:
print(len(gdf), len(all_state_merged))

11947 11947


In [189]:
print(len(vaCong16), len(vaCong18), len(vaMerged))

2446 2446 2446


In [190]:
print(len(riCong16), len(riCong18), len(riMerged))

419 419 419


In [191]:
print(len(txCong16), len(txCong18), len(txMerged))

9082 9082 9082


# merge cong to gdf

In [192]:
gdf = all_state_merged.merge(gdf, left_on="CName", right_on="CName")
gdf.head(2)

,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Other,Cong16Total,CName,Cong16Grn,Cong18Dem,Cong18Rep,Cong18Lib,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
0,955.0,627.0,0.0,3.0,1691.0,ri-bristol-0101,0.0,916,494,0,...,3268,3055,23,3,110,73,None,None,None,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."
1,1225.0,790.0,0.0,3.0,2167.0,ri-bristol-0102,0.0,1245,558,0,...,2921,2740,18,3,85,70,None,None,None,"MULTIPOLYGON (((-71.35581 41.74533, -71.35560 ..."


In [193]:
gdf.columns

Index(['Cong16Dem', 'Cong16Rep', 'Cong16Lib', 'Cong16Other', 'Cong16Total',
       'CName', 'Cong16Grn', 'Cong18Dem', 'Cong18Rep', 'Cong18Lib',
       'Cong18Other', 'Cong18Total', 'dist_num', 'Cong18Grn', 'County',
       'Precinct #', 'Hillary Clinton/Dem', 'Donald J. Trump/Rep',
       'Gary Johnson/Lib', 'Jill Stein/Grn', 'Other/Other', 'Total Votes',
       'State', 'Neighbors', 'Total Population', 'White',
       'Black or African American', 'American Indian', 'Asian',
       'Other Population', 'MissingVote', 'MissingDemo', 'VoteDemoConflict',
       'geometry'],
      dtype='object')

In [194]:
mapper = {"Hillary Clinton/Dem":"Pres16Dem", 'Donald J. Trump/Rep':"Pres16Rep",
         "Gary Johnson/Lib":"Pres16Lib", "Jill Stein/Grn":"Pres16Grn",
         "Other/Other":"Pres16Other", "Total Votes":"Pres16Total"}
gdf.rename(mapper, inplace=True, axis=1)
gdf.head(1)

,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Other,Cong16Total,CName,Cong16Grn,Cong18Dem,Cong18Rep,Cong18Lib,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
0,955.0,627.0,0.0,3.0,1691.0,ri-bristol-0101,0.0,916,494,0,...,3268,3055,23,3,110,73,None,None,None,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."


In [195]:
copy = gdf.copy()

In [196]:
len(gdf)

11947

# 19. Identify anomalous precinct data (required) (sequence diagram not required)
- At a minimum, precincts should be considered anomalous if the underlying data is not consistent.
- For example, population or voting data could be zero or voting data might not be a reasonable percentage of population. This is considered a distinct error category, so you should store the associated data in the errors data structure for subsequent addition to the DB. 

In [206]:
gdf = copy.copy()

In [207]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11947 entries, 0 to 11946
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Cong16Dem                  11947 non-null  float64 
 1   Cong16Rep                  11947 non-null  float64 
 2   Cong16Lib                  11947 non-null  float64 
 3   Cong16Other                11947 non-null  float64 
 4   Cong16Total                11947 non-null  float64 
 5   CName                      11947 non-null  object  
 6   Cong16Grn                  11947 non-null  float64 
 7   Cong18Dem                  11947 non-null  int64   
 8   Cong18Rep                  11947 non-null  int64   
 9   Cong18Lib                  11947 non-null  int64   
 10  Cong18Other                11947 non-null  int64   
 11  Cong18Total                11947 non-null  int64   
 12  dist_num                   11947 non-null  object  
 13  Cong18Grn                  1194

In [208]:
'''
Categories: MissingVote, MissingDemo, VoteDemoConflict 
'''

gdf["MissingVote"] = ""
gdf["MissingDemo"] = ""
gdf["VoteDemoConflict"] = ""
MPV = 0
MC16V = 0
MC18V = 0
MD = 0
PVDC = 0
C16VDC = 0
C18VDC = 0
counter = 0

gdf = copy.copy()
for index, row in gdf.iterrows():
    if counter%1000==0:
        print("#", end="")
    counter += 1
    
    total_pres = int(row["Pres16Total"])
    total_cong16 = int(row["Cong16Total"])
    total_cong18 = int(row["Cong18Total"])
    total_demo = row["Total Population"]
    
    check_conflict_flag = 1
    to_add = ""
    if total_pres == 0:
        to_add = "p16"
        check_conflict_flag = 0
        MPV += 1
    if total_cong16 == 0:
        if to_add != "":
            to_add += ";c16"
        else:
            to_add = "c16"
        check_conflict_flag = 0
        MC16V += 1
    if total_cong18 == 0:
        if to_add != "":
            to_add += ";c18"
        else:
            to_add = "c18"
        check_conflict_flag = 0
        MC18V += 1
        
    gdf.at[index, "MissingVote"] = to_add
    
    
    if total_demo == 0:
        gdf.at[index, "MissingDemo"] = "MissingDemo"
        check_conflict_flag = 0
        MD += 1
    else:
        gdf.at[index, "MissingDemo"] = ""
    
    to_add = ""
    if check_conflict_flag:
        if total_pres < total_demo*0.3 or total_pres > total_demo*0.75:
            to_add = "p16"
            PVDC += 1
        if total_cong16 < total_demo*0.2 or total_cong16 > total_demo*0.8:
            if to_add != "":
                to_add += ";c16"
            else:
                to_add = "c16"
            C16VDC += 1
        if total_cong18 < total_demo*0.2 or total_cong18 > total_demo*0.8:
            if to_add != "":
                to_add += ";c18"
            else:
                to_add = "c18"
            C18VDC += 1
            
    gdf.at[index, "VoteDemoConflict"] = to_add

    
print("\nMissing Pres Vote:", MPV)
print("Missing Cong16 Vote:", MC16V)
print("Missing Cong18 Vote:", MC18V)
print("Missing Demo:", MD)
print("Pres Vote Demo Conflict:", PVDC)
print("Cong16 Vote Demo Conflict:", C16VDC)
print("Cong18 Vote Demo Conflict:", C18VDC)

############
Missing Pres Vote: 308
Missing Cong16 Vote: 690
Missing Cong18 Vote: 508
Missing Demo: 902
Pres Vote Demo Conflict: 5626
Cong16 Vote Demo Conflict: 2760
Cong18 Vote Demo Conflict: 3077


In [209]:
gdf.groupby(["VoteDemoConflict"]).count()

,Cong16Dem,Cong16Rep,Cong16Lib,Cong16Other,Cong16Total,CName,Cong16Grn,Cong18Dem,Cong18Rep,Cong18Lib,...,Neighbors,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,geometry
VoteDemoConflict,,,,,,,,,,,,,,,,,,,,,
,6143,6143,6143,6143,6143,6143,6143,6143,6143,6143,...,6143,6143,6143,6143,6143,6143,6143,6143,6143,6143
c16,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
c16;c18,53,53,53,53,53,53,53,53,53,53,...,53,53,53,53,53,53,53,53,53,53
c18,114,114,114,114,114,114,114,114,114,114,...,114,114,114,114,114,114,114,114,114,114
p16,2452,2452,2452,2452,2452,2452,2452,2452,2452,2452,...,2452,2452,2452,2452,2452,2452,2452,2452,2452,2452
p16;c16,264,264,264,264,264,264,264,264,264,264,...,264,264,264,264,264,264,264,264,264,264
p16;c16;c18,2432,2432,2432,2432,2432,2432,2432,2432,2432,2432,...,2432,2432,2432,2432,2432,2432,2432,2432,2432,2432
p16;c18,478,478,478,478,478,478,478,478,478,478,...,478,478,478,478,478,478,478,478,478,478


In [210]:
gdf = gpd.GeoDataFrame(gdf)

In [212]:
gdf.to_file("all_states_precincts_cname_neighbor_demo_OG_anomalous_fixNeighborVA_congVote.json", driver="GeoJSON")

# parse

In [8]:
gdf = gpd.read_file("all_states_precincts_cname_neighbor_demo_OG_anomalous_fixNeighborVA_congVote.json")
gdf.head(2)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,...,Total Population,White,Black or African American,American Indian,Asian,Other Population,MissingVote,MissingDemo,VoteDemoConflict,geometry
0,Bristol,0101,1027,522,60,23,2,1634,ri,ri-bristol-0101,...,3268,3055,23,3,110,73,None,None,None,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."
1,Bristol,0102,1331,683,68,21,3,2106,ri,ri-bristol-0102,...,2921,2740,18,3,85,70,None,None,None,"MULTIPOLYGON (((-71.35581 41.74533, -71.35560 ..."


In [213]:
c = list(gdf.columns)
c.remove("CName")
c.remove("State")
c.remove("MissingVote")
c.remove("MissingDemo")
c.remove("VoteDemoConflict")
c.remove("geometry")
gdf.drop(columns=c, inplace=True, axis=1)
gdf.head(1)

,CName,State,MissingVote,MissingDemo,VoteDemoConflict,geometry
0,ri-bristol-0101,ri,,,,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."


In [214]:
newgdf = pd.DataFrame(columns=["State", "CName", "MissingVote", "MissingDemo", "VoteDemoConflict", "x_pos", "y_pos"])
newgdf

,State,CName,MissingVote,MissingDemo,VoteDemoConflict,x_pos,y_pos


In [215]:
for index, row in gdf.iterrows():
    x, y = "", ""
    if row["geometry"].geom_type == "MultiPolygon":
        biggest_area = 0
        for shape in row["geometry"]:
            if shape.area > biggest_area:
                x, y = shape.centroid.coords[0] 
                biggest_area = shape.area
    else:
        x, y = row["geometry"].centroid.coords[0]
        
    newgdf = newgdf.append({"State":row["State"], "CName":row["CName"], "MissingVote":row["MissingVote"],
                           "MissingDemo":row["MissingDemo"], "VoteDemoConflict":row["VoteDemoConflict"],
                           "x_pos":x, "y_pos":y}, ignore_index=True)

In [216]:
newgdf.head()

,State,CName,MissingVote,MissingDemo,VoteDemoConflict,x_pos,y_pos
0,ri,ri-bristol-0101,,,,-71.330025,41.756734
1,ri,ri-bristol-0102,,,,-71.335621,41.745035
2,ri,ri-bristol-0103,,,,-71.319870,41.732146
3,ri,ri-bristol-0104,,,,-71.305247,41.759840
4,ri,ri-bristol-0105,,,,-71.300874,41.720414


In [217]:
for index, row in newgdf.iterrows():
    if row["MissingVote"] == "" and row["MissingDemo"] == "" and row["VoteDemoConflict"] == "":
        newgdf.drop(index=index, inplace=True)
newgdf.head()

,State,CName,MissingVote,MissingDemo,VoteDemoConflict,x_pos,y_pos
5,ri,ri-bristol-0201,,,p16;c16;c18,-71.266333,41.708733
11,ri,ri-bristol-0207,,,p16;c16;c18,-71.256753,41.661750
14,ri,ri-providence-0401,,,p16;c16;c18,-71.385925,41.887850
15,ri,ri-providence-0402,,,p16;c18,-71.389906,41.892830
16,ri,ri-providence-0403,,,c18,-71.392898,41.885401


In [218]:
newgdf.to_csv("anomalous_data.csv", index=False)